In [3]:
!pip install torch==2.4.0+cu121 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 132.7 MB/s  0:00:0400:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 139.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 135.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 139.3 MB/s  0:00:0300:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 140.6 MB/s  0:00:0200:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 168.7 MB/s  0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 144.9 MB/s  0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 160.4 MB/s  0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 172.3 MB/s  0:00:0100:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 172.9 MB/s

In [4]:
#!/usr/bin/env python3
"""
CUDA Version Forensics Experiment
Tests if different CUDA toolkit versions create detectable activation deviations
Same PyTorch version (2.4.0), different CUDA toolkits (11.8 vs 12.0)
"""

import os
os.environ['HF_HOME'] = '/workspace/huggingface_cache'
os.environ['TRANSFORMERS_CACHE'] = '/workspace/huggingface_cache'
os.environ['HF_DATASETS_CACHE'] = '/workspace/huggingface_cache'

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc
import numpy as np
from datetime import datetime
import json
import socket

print("="*60)
print("CUDA VERSION FORENSICS EXPERIMENT")
print("="*60)

# System info
HOSTNAME = socket.gethostname()
CONTAINER_ID = os.environ.get('HOSTNAME', 'unknown')

print(f"\nSystem Info:")
print(f"  Hostname: {HOSTNAME}")
print(f"  Container: {CONTAINER_ID}")
print(f"  GPU: {torch.cuda.get_device_name(0)}")
print(f"  PyTorch: {torch.__version__}")
print(f"  CUDA Runtime: {torch.version.cuda}")

# Extract CUDA version from PyTorch build
pytorch_version = torch.__version__
if '+cu' in pytorch_version:
    cuda_build = pytorch_version.split('+cu')[1].split('.')[0]  # e.g., "118" or "120"
    cuda_version_str = f"cu{cuda_build}"
    print(f"  CUDA Build: {cuda_version_str}")
else:
    print(f"  WARNING: Could not detect CUDA version from PyTorch build")
    cuda_version_str = "unknown"

print()

# Check GPU memory
mem_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
mem_allocated = torch.cuda.memory_allocated(0) / 1024**3
print(f"GPU Memory:")
print(f"  Total: {mem_total:.1f} GB")
print(f"  Currently allocated: {mem_allocated:.2f} GB")
print()

def collect_activations_multilayer(model, tokenizer, prompt, device="cuda"):
    """Extract hidden states from multiple layers"""
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
    gc.collect()
    
    inputs = tokenizer([prompt], return_tensors="pt", padding=True)
    seq_len = inputs['input_ids'].shape[1]
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, use_cache=False)
    
    last_valid_pos = inputs['attention_mask'][0].sum() - 1
    num_layers = len(outputs.hidden_states) - 1
    
    # Sample layers: dense early, then sparse
    layer_indices = [1, 2, 3, 4, 7, 10, 14, 18, 22, num_layers]
    
    activations = {}
    for idx in layer_indices:
        layer_activation = outputs.hidden_states[idx][0, last_valid_pos, :].cpu().clone()
        activations[f"layer_{idx}"] = layer_activation
    
    # Aggressive cleanup
    del outputs.hidden_states
    del outputs
    del inputs
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    return activations, seq_len, layer_indices

# Setup
CACHE_DIR = '/workspace/huggingface_cache'
model_name = "Qwen/Qwen2.5-7B-Instruct"

print(f"Loading {model_name} in BF16...")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    cache_dir=CACHE_DIR,
    low_cpu_mem_usage=True,
    device_map="auto",
    attn_implementation="eager"
)

# Check what attention implementation is actually being used
config_impl = getattr(model.config, '_attn_implementation', 'not set')
print(f"Model attention implementation: {config_impl}")

# Load prompt from file, or use default
prompt_file = "dummytext.txt"
try:
    with open(prompt_file, 'r', encoding='utf-8') as f:
        prompt = f.read().strip()
    print(f"✓ Loaded prompt from {prompt_file}")
except FileNotFoundError:
    prompt = """The development of large language models has fundamentally transformed natural language processing 
and artificial intelligence more broadly. These models, trained on vast corpora of text data, have demonstrated 
remarkable capabilities across a wide range of tasks, from translation and summarization to question answering 
and creative writing. However, their deployment raises significant challenges related to computational efficiency, 
interpretability, and safety."""
    print(f"⚠ Using default prompt")

prompt_tokens = len(tokenizer.encode(prompt))
print(f"Prompt tokens: {prompt_tokens}")
print()

# Run experiment
num_repetitions = 5
results = {}
all_activations = {}
layer_indices = None

print(f"{'='*60}")
print(f"CUDA VERSION COMPARISON EXPERIMENT")
print(f"CUDA Build: {cuda_version_str}")
print(f"Model: {model_name}")
print(f"Precision: BF16 (bfloat16)")
print(f"Attention: flash_attention_2")
print(f"Prompt tokens: {prompt_tokens}")
print(f"Repetitions: {num_repetitions}")
print(f"{'='*60}\n")

print(f"\nCollecting multi-layer activations ({num_repetitions} repetitions)...")

for rep in range(num_repetitions):
    mem_before = torch.cuda.memory_allocated(0) / 1024**3
    
    activations, seq_len, layer_idx_list = collect_activations_multilayer(
        model, tokenizer, prompt, device="cuda"
    )
    
    mem_after = torch.cuda.memory_allocated(0) / 1024**3
    
    if layer_indices is None:
        layer_indices = layer_idx_list
        print(f"  Extracting from layers: {layer_indices}")
        for layer_name in activations.keys():
            results[layer_name] = []
    
    # Store activations for each layer
    for layer_name, activation in activations.items():
        results[layer_name].append(activation)
    
    if rep == 0:
        print(f"  Rep 0 norms: {', '.join([f'{k}={torch.norm(v).item():.2f}' for k, v in activations.items()])}")
        print(f"  Rep 0 memory: before={mem_before:.2f}GB, after={mem_after:.2f}GB")
    
    # Cleanup
    del activations
    gc.collect()
    torch.cuda.empty_cache()

print(f"  ✓ Completed all repetitions\n")

# Stack repetitions into tensors
for layer_name in results.keys():
    results[layer_name] = torch.stack(results[layer_name])

# Check repeatability
last_layer_name = f"layer_{layer_indices[-1]}"
first_rep = results[last_layer_name][0]
all_identical = all(
    torch.equal(first_rep, results[last_layer_name][i]) 
    for i in range(1, num_repetitions)
)
print(f"Repeatability (last layer): {'✓ All identical' if all_identical else '⚠ Varies'}")

# Convert to numpy for JSON storage
for layer_name, tensor in results.items():
    all_activations[layer_name] = tensor.float().numpy().tolist()

# Compute statistics
layer_stats = {}
for layer_name in results.keys():
    mean_activation = results[layer_name].mean(dim=0)
    layer_stats[layer_name] = {
        "mean_norm": float(torch.norm(mean_activation)),
        "std_within_reps": float(torch.stack([
            torch.norm(results[layer_name][i] - mean_activation) 
            for i in range(num_repetitions)
        ]).std())
    }

print("\nLayer activation norms:")
for layer_name in sorted(layer_stats.keys(), key=lambda x: int(x.split('_')[1])):
    norm = layer_stats[layer_name]["mean_norm"]
    print(f"  {layer_name}: {norm:.2f}")

# Save results
output = {
    "experiment": "cuda_version_forensics",
    "timestamp": datetime.now().isoformat(),
    "model": model_name,
    "hardware": {
        "gpu": torch.cuda.get_device_name(0),
        "hostname": HOSTNAME,
        "container_id": CONTAINER_ID
    },
    "software": {
        "pytorch_version": torch.__version__,
        "cuda_runtime": torch.version.cuda,
        "cuda_build": cuda_version_str,
        "attention_implementation": config_impl
    },
    "config": {
        "dtype": "bfloat16",
        "prompt_tokens": prompt_tokens,
        "repetitions": num_repetitions,
        "layers_sampled": layer_indices,
        "hidden_dim": int(results[last_layer_name][0].shape[0])
    },
    "layer_statistics": layer_stats,
    "reproducibility": {
        "all_repetitions_identical": all_identical
    },
    "raw_activations": all_activations
}

# Filename includes CUDA version
gpu_name = torch.cuda.get_device_name(0).replace(' ', '_')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f"{gpu_name}_{cuda_version_str}_7b_bf16_eager_{timestamp}.json"
output_path = f"/workspace/experiments/{output_file}"

with open(output_path, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Results saved to {output_path}")
print(f"✓ File size: ~{len(json.dumps(output)) / 1024:.1f} KB")

print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print(f"CUDA Build: {cuda_version_str}")
print("="*60)
print("\nNext steps:")
print("1. If this was cu118: Restart kernel, install cu120, run again")
print("2. If this was cu120: Compare the two JSON files")
print("="*60)

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


CUDA VERSION FORENSICS EXPERIMENT

System Info:
  Hostname: 75a750b4edaf
  Container: 75a750b4edaf
  GPU: NVIDIA A100 80GB PCIe
  PyTorch: 2.4.0+cu121
  CUDA Runtime: 12.1
  CUDA Build: cu121

GPU Memory:
  Total: 79.3 GB
  Currently allocated: 0.00 GB

Loading Qwen/Qwen2.5-7B-Instruct in BF16...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model attention implementation: eager
✓ Loaded prompt from dummytext.txt
Prompt tokens: 3404

CUDA VERSION COMPARISON EXPERIMENT
CUDA Build: cu121
Model: Qwen/Qwen2.5-7B-Instruct
Precision: BF16 (bfloat16)
Attention: flash_attention_2
Prompt tokens: 3404
Repetitions: 5


  Extracting from layers: [1, 2, 3, 4, 7, 10, 14, 18, 22, 28]
  Rep 0 norms: layer_1=8.62, layer_2=14.81, layer_3=17.12, layer_4=18.50, layer_7=35.50, layer_10=55.50, layer_14=66.00, layer_18=79.00, layer_22=160.00, layer_28=312.00
  Rep 0 memory: before=14.19GB, after=14.20GB
  ✓ Completed all repetitions

Repeatability (last layer): ✓ All identical

Layer activation norms:
  layer_1: 8.62
  layer_2: 14.81
  layer_3: 17.12
  layer_4: 18.50
  layer_7: 35.50
  layer_10: 55.50
  layer_14: 66.00
  layer_18: 79.00
  layer_22: 160.00
  layer_28: 312.00

✓ Results saved to /workspace/experiments/NVIDIA_A100_80GB_PCIe_cu121_7b_bf16_eager_20251103_140554.json
✓ File size: ~2216.7 KB

EXPERIMENT COMPLETE
CUDA Build: cu121

Next